In [1]:
def get_rotations(x, y, z):
    return [
        (x, y, z),
        (-y, x, z),
        (-x, -y, z),
        (y, -x, z),
    ]

def get_z_orientations(x, y, z):
    return [
        (x, y, z),
        (x, z, -y),
        (x, -y, -z),
        (x, -z, y),
        (-z, y, x),
        (z, y, -x),
    ]
    
def get_orientations(x, y, z):
    for xi, yi, zi in get_z_orientations(x, y, z):
        yield from get_rotations(xi, yi, zi)

In [2]:
from itertools import product

class Scanner:
    def __init__(self, n, positions):
        self.n = n
        self.orientations = [[] for i in range(24)]
        for x, y, z in positions:
            for p, pos in enumerate(get_orientations(x, y, z)):
                self.orientations[p].append(pos)
        self.fixed_beacons = None
        self.pos = None
        
    def __repr__(self):
        return f"<Scanner {self.n}>"
        
    def pairing_mode(self, scanners):
        if self.fixed_beacons is not None:
            return
        for scanner in scanners:
            if scanner.fixed_beacons is None:
                continue
            if self.find_overlap(scanner):
                return
                
    def find_overlap(self, other):
        for orientation in self.orientations:
            for fx, fy, fz in other.fixed_beacons:
                for bx, by, bz in orientation:
                    dx, dy, dz = (fx - bx, fy - by, fz - bz)
                    shifted_beacons = {(x + dx, y + dy, z + dz) for x, y, z in orientation}
                    common_beacons = shifted_beacons & other.fixed_beacons
                    if len(common_beacons) >= 12:
                        self.fixed_beacons = shifted_beacons
                        ox, oy, oz = other.pos
                        self.pos = (dx, dy, dz)
                        return True
                    
    def distance_to(self, other):
        ax, ay, az = self.pos
        bx, by, bz = other.pos
        return abs(ax - bx) + abs(ay - by) + abs(az - bz)

In [3]:
scanners = []
with open('input') as f:
    for n, block in enumerate(f.read().split('\n\n')):
        lines = block.split('\n')
        positions = []
        for line in lines[1:]:
            pos = tuple(int(i) for i in line.strip().split(',') if line.strip())
            if pos:
                positions.append(pos)
        scanners.append(Scanner(n, positions))

s = scanners[0]
s.fixed_beacons = set(s.orientations[0])
s.pos = (0, 0, 0)

In [4]:
while len([s for s in scanners if s.fixed_beacons is None]):
    for scanner in scanners:
        scanner.pairing_mode(scanners)

In [5]:
beacons = set()
for scanner in scanners:
    beacons |= scanner.fixed_beacons
        
print("Part 1:")
print(len(beacons))

Part 1:
430


In [6]:
from itertools import permutations

print("Part 2:")
print(max(a.distance_to(b) for a, b in permutations(scanners, 2)))

Part 2:
11860
